In [85]:
# 구글 드라이브 연동
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [355]:
%cd /content/drive/MyDrive/
!git clone https://github.com/oglee815/mygpt-lecture.git
%cd /content/drive/MyDrive/mygpt-lecture

/content/drive/MyDrive
Cloning into 'mygpt-lecture'...
remote: Enumerating objects: 234, done.
remote: Counting objects: 100% (234/234), done.
remote: Compressing objects: 100% (202/202), done.
remote: Total 234 (delta 60), reused 166 (delta 28), pack-reused 0
Receiving objects: 100% (234/234), 5.22 MiB | 10.67 MiB/s, done.
Resolving deltas: 100% (60/60), done.
/content/drive/MyDrive/mygpt-lecture


# Stage 1. 도메인 특화 사전학습 데이터를 만들어보자

### Domain Adaptive Pretraining 데이터 만들기 (100% 수작업)

In [356]:
import pandas as pd

In [90]:
# 원하는 데이터 읽기
# 본인이 직접 만든 데이터를 읽어오세요!!
data = pd.read_csv("/content/drive/MyDrive/mygpt-lecture/data/stage1. domain_adaptive_pretraining/아이브.csv")

In [91]:
data.head()

,주제,내용,길이
0,아이브의 개요,2021년 12월 1일에 데뷔한 스타쉽엔터테인먼트 소속의 대한민국 6인조 다국적 걸...,138
1,아이브의 멤버,"안유진, 가을, 레이, 장원영, 리즈, 이서",24
2,아이브의 콘셉트1,청순 콘셉트를 지나 다시금 걸 크러시 콘셉트가 트렌드로 돌아온 4세대 걸그룹답게 I...,405
3,아이브의 콘셉트2,사실 이 차별화는 현재 걸크러시 콘셉트로 활동하는 걸그룹 대부분에게 해당되는 말이긴...,380
4,아이브의 콘셉트3,연타석 홈런을 친 데뷔 싱글 1집 타이틀곡 <ELEVEN>과 싱글 2집 타이틀곡 <...,269


In [92]:
len(data)

25

In [94]:
# 주제를 학습을 위한 Prompt 형식으로 변환합니다.
def prompting(topic):
    return f"다음 주제에 대하여, 위키피디아에 있는 정보를 어떠한 수정 없이 사실대로 말해주세요.\n주제:{topic}\n"

In [95]:
# 학습에 사요되는 포맷인 jsonl 타입으로 변경합니다
instruction = []
for a in data.iterrows():
    topic = a[1]['주제']
    output = a[1]['내용']
    instruction.append({'instruction': prompting(topic), 'output': output})

In [108]:
# 알맞은 경로에 알맞은 이름으로 저장
import json
with open("/content/drive/MyDrive/mygpt-lecture/data/stage1. domain_adaptive_pretraining/domain_adaptive_ive_pretrain.jsonl" , encoding= "utf-8",mode="w") as file:
	for i in instruction: file.write(json.dumps(i, ensure_ascii=False) + "\n")

# API test

In [98]:
!pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.5/75.5 kB 1.5 MB/s eta 0:00:00


In [99]:
import openai

In [101]:
openai.api_version=""
openai.api_key="### 여기에 본인의 키를 입력하세요 ###"
response = openai.ChatCompletion.create(
    model='gpt-3.5-turbo',
      messages=[
        {"role": "user", "content": "안녕? 넌 이름이 뭐니?"}],
    max_tokens=512,
    temperature=0,
)

print(response['choices'][0]['message']['content'])

안녕! 나는 OpenAI의 GPT-3 모델이야. 이름은 없어, 그냥 GPT-3라고 불러줘! 어떻게 도와줄까?


# Stage1 데이터 생성하기, ChatGPT를 활용하여 도메인 특화 질의 응답 생성



In [109]:
# 자신이 만든 데이터 읽어오기!!

data=[]
for a in open("/content/drive/MyDrive/mygpt-lecture/data/stage1. domain_adaptive_pretraining/domain_adaptive_ive_pretrain.jsonl", 'r'):
    data.append(json.loads(a))

In [113]:
print(data[0]['instruction'])
print(data[0]['output'])

다음 주제에 대하여, 위키피디아에 있는 정보를 어떠한 수정 없이 사실대로 말해주세요.
주제:아이브의 개요

2021년 12월 1일에 데뷔한 스타쉽엔터테인먼트 소속의 대한민국 6인조 다국적 걸그룹이다.[2]

그룹명인 IVE는 'I HAVE'의 축약형인 'I'VE'에서 유래되었으며, "우리가 가진 것들을 모두 당당히 보여 드리겠다."는 포부를 담고 있다.


In [114]:
from tqdm import tqdm

In [115]:
qa_data = []
error_data = []

In [131]:
# ChatGPT에게 여러번 물어보면 그만큼 돈과 시간이 많이 들겠죠?
# 따라서 10개의 질문/정답 쌍을 한번에 받습니다!

PROMPT="다음 주제와 내용을 보고 답변 할 수 있는 질문과 정답 쌍을 10개 만들어줘.\n주제:{title}\n내용:{content}\n\
답변은 반드시 아래 형태를 띄어야 해.\n\
Q1:질문\n\
A1:정답\n\
\n\
Q2:질문\n\
A2:정답\n\
\n\
Q3:질문\n\
A3:정답\n\
\n\
Q4:질문\n\
A4:정답\n\
\n\
Q5:질문\n\
A5:정답\n\
...\n\
Q10:질문\n\
A10:정답\n\
"

In [117]:
# ChatGPT API를 통해 데이터 얻어오기
# 중간에 에러가 나면 error_data에 넣어두고, 아래 for_loop의 data -> error_data 로 고쳐서 다시 돌려도 됨!!

for a in tqdm(data):
    title = a['instruction'].split("주제:")[-1]
    content = a['output']
    try:
        response = openai.ChatCompletion.create(
        model='gpt-3.5-turbo',
        messages=[
            {'role':"system","content":"You are a helpful assistant."},
            {'role':"user","content":PROMPT.format_map({'title':title, 'content':content})},
            ]
        )
        qa_data.append(response['choices'][0]['message']['content'])
    except Exception as e:
        error_data.append(a)
        print(title)
        print(e)

100%|██████████| 25/25 [08:01<00:00, 19.27s/it]


In [133]:
# chatgpt의 응답에서 Q와 A를 발라냅니다.
instruction_data=[]
for a in qa_data:
    try:
        for i, b in enumerate(a.split("\n\n")):
            i +=1
            q = b.split("Q"+str(i)+":")[1].split("\nA"+str(i)+":")[0].lstrip()
            ans = b.split("A"+str(i)+":")[1].split("\n\nQ"+str(i)+":")[0].lstrip()
            instruction_data.append({'instruction':q,'output':ans})
    except Exception as e:
        print(e)
        continue

In [134]:
len(instruction_data)

248

In [123]:
# 잘 만들어졌는지 볼까요?
for a in instruction_data:
    print(a)

{'instruction': '아이브는 어떤 소속사에 소속되어 있나요?', 'output': '아이브는 스타쉽엔터테인먼트에 소속되어 있습니다.'}
{'instruction': '아이브는 몇 명으로 이루어져 있나요?', 'output': '아이브는 6인조로 이루어져 있습니다.'}
{'instruction': '아이브의 데뷔일은 언제인가요?', 'output': '아이브의 데뷔일은 2021년 12월 1일입니다.'}
{'instruction': 'IVE의 그룹명은 어떤 의미를 가지고 있나요?', 'output': 'IVE는 \'I HAVE\'의 축약형인 \'I\'VE\'에서 유래되었으며, "우리가 가진 것들을 모두 당당히 보여 드리겠다."는 포부를 담고 있습니다.'}
{'instruction': '아이브는 어떤 국적을 가진 걸그룹인가요?', 'output': '아이브는 다국적 걸그룹입니다.'}
{'instruction': '아이브의 소속사는 어떤 엔터테인먼트 기업인가요?', 'output': '아이브의 소속사는 스타쉽엔터테인먼트입니다.'}
{'instruction': '아이브의 데뷔 후 첫 음악 프로그램 1위는 언제였나요?', 'output': '아이브의 데뷔 후 첫 음악 프로그램 1위는 공개되지 않았습니다.'}
{'instruction': '아이브의 멤버는 누구들인가요?', 'output': '아이브의 멤버는 공개되지 않았습니다.'}
{'instruction': '아이브의 음악 스타일은 어떤 특징이 있나요?', 'output': '아이브의 음악 스타일은 아직 공개되지 않았습니다.'}
{'instruction': '아이브는 데뷔 후 어떤 활동을 할 예정인가요?', 'output': '아이브의 데뷔 후 활동 계획은 아직 공개되지 않았습니다.'}
{'instruction': '아이브의 멤버는 누구인가요?', 'output': '안유진, 가을, 레이, 장원영, 리즈, 이서'}
{'instruction': '안유진은 어떤 역할을 맡고 있나요?', 'output': '안유진

In [135]:
# Train/Test로 적당히 나눕니다.
import random
test_data=[]
train_data=[]

for a in instruction_data:
    if random.random() < 0.1:
        test_data.append(a)
    else:
        train_data.append(a)

In [136]:
# 알맞은 이름으로 저장
import json
with open("/content/drive/MyDrive/mygpt-lecture/data/stage1. domain_instruction_tuning/ive_instruction_train.jsonl" , encoding= "utf-8",mode="w") as file:
	for i in train_data: file.write(json.dumps(i , ensure_ascii=False) + "\n")

with open("/content/drive/MyDrive/mygpt-lecture/data/stage1. domain_instruction_tuning/ive_instruction_test.jsonl" , encoding= "utf-8",mode="w") as file:
	for i in test_data: file.write(json.dumps(i, ensure_ascii=False) + "\n")

# Stage 2의 RM 데이터 생성

### Data Format

data = [{'prompt':"질문", 'chosen':"더 우수한 답변", 'reject':"덜 우수한 답변"}]

In [138]:
# 자신이 만든 사전학습 데이터 읽어오기!!

data=[]
for a in open("/content/drive/MyDrive/mygpt-lecture/data/stage1. domain_adaptive_pretraining/domain_adaptive_ive_pretrain.jsonl", 'r'):
    data.append(json.loads(a))

### 좋은 답변 만들기

ChatGPT에게 지문을 주고, 지문을 보고 만들어 낼 수 있는 질문과 그에 따른 정답도 물어봅니다!

(Stage 1 데이터 생성 방식과 동일하나, temperature만 변경)

In [139]:
import time
from tqdm import tqdm

qa_data = []
error_data = []

In [ ]:
PROMPT="다음 주제와 내용을 보고 답변 할 수 있는 질문과 정답 쌍을 10개 만들어줘.\n주제:{title}\n내용:{content}\n\
답변은 반드시 아래 형태를 띄어야 해.\n\
Q1:질문\n\
A1:정답\n\
\n\
Q2:질문\n\
A2:정답\n\
\n\
Q3:질문\n\
A3:정답\n\
\n\
Q4:질문\n\
A4:정답\n\
\n\
Q5:질문\n\
A5:정답\n\
...\n\
Q10:질문\n\
A10:정답\n\
"

In [141]:
for a in tqdm(data):

    title = a['instruction'].split("주제:")[-1]
    content = a['output']
    try:
        response = openai.ChatCompletion.create(
            model='gpt-3.5-turbo',
            messages=[
                    {'role':"system","content":"You are a helpful assistant."},
                    {'role':"user","content":PROMPT.format_map({'title':title, 'content':content})},
                ],
            temperature=1
        )
        qa_data.append(response['choices'][0]['message']['content'])
        time.sleep(1)
    except Exception as e:
        error_data.append(a)
        print(title)
        print(e)

100%|██████████| 25/25 [08:28<00:00, 20.35s/it]


In [144]:
print(qa_data[0])

Q1: 아이브는 어떤 소속사에 소속되어 있나요?
A1: 아이브는 스타쉽엔터테인먼트에 소속되어 있습니다.

Q2: 아이브는 몇 명으로 구성되어 있나요?
A2: 아이브는 6인조로 구성되어 있습니다.

Q3: 아이브의 데뷔일은 언제인가요?
A3: 아이브의 데뷔일은 2021년 12월 1일입니다.

Q4: 아이브의 그룹명은 어디에서 유래되었나요?
A4: 아이브의 그룹명은 'I HAVE'의 축약형인 'I'VE'에서 유래되었습니다.

Q5: 아이브의 그룹명은 어떤 포부를 담고 있나요?
A5: 아이브의 그룹명은 "우리가 가진 것들을 모두 당당히 보여 드리겠다."는 포부를 담고 있습니다.

Q6: 아이브는 어느 국적의 그룹인가요?
A6: 아이브는 다국적 그룹입니다.

Q7: 아이브의 멤버들은 누가 있나요?
A7: 아이브의 멤버들은 곧 발표될 예정입니다.

Q8: 아이브의 소속사에는 어떤 다른 그룹들이 있나요?
A8: 스타쉽엔터테인먼트에는 몬스타엑스, 크래비티 등 다양한 그룹들이 있습니다.

Q9: 아이브는 데뷔 후 어떤 음악 활동을 펼칠 예정인가요?
A9: 아이브는 데뷔 후 음악 활동뿐만 아니라 다양한 활동을 펼칠 예정입니다.

Q10: 아이브의 데뷔 앨범에는 어떤 곡들이 수록되어 있나요?
A10: 아이브의 데뷔 앨범의 곡 목록은 아직 공개되지 않았습니다.


In [278]:
qa_data[0]

'Q1: 아이브는 어떤 소속사에 소속되어 있나요?\nA1: 아이브는 스타쉽엔터테인먼트에 소속되어 있습니다.\n\nQ2: 아이브는 몇 명으로 구성되어 있나요?\nA2: 아이브는 6인조로 구성되어 있습니다.\n\nQ3: 아이브의 데뷔일은 언제인가요?\nA3: 아이브의 데뷔일은 2021년 12월 1일입니다.\n\nQ4: 아이브의 그룹명은 어디에서 유래되었나요?\nA4: 아이브의 그룹명은 \'I HAVE\'의 축약형인 \'I\'VE\'에서 유래되었습니다.\n\nQ5: 아이브의 그룹명은 어떤 포부를 담고 있나요?\nA5: 아이브의 그룹명은 "우리가 가진 것들을 모두 당당히 보여 드리겠다."는 포부를 담고 있습니다.\n\nQ6: 아이브는 어느 국적의 그룹인가요?\nA6: 아이브는 다국적 그룹입니다.\n\nQ7: 아이브의 멤버들은 누가 있나요?\nA7: 아이브의 멤버들은 곧 발표될 예정입니다.\n\nQ8: 아이브의 소속사에는 어떤 다른 그룹들이 있나요?\nA8: 스타쉽엔터테인먼트에는 몬스타엑스, 크래비티 등 다양한 그룹들이 있습니다.\n\nQ9: 아이브는 데뷔 후 어떤 음악 활동을 펼칠 예정인가요?\nA9: 아이브는 데뷔 후 음악 활동뿐만 아니라 다양한 활동을 펼칠 예정입니다.\n\nQ10: 아이브의 데뷔 앨범에는 어떤 곡들이 수록되어 있나요?\nA10: 아이브의 데뷔 앨범의 곡 목록은 아직 공개되지 않았습니다.'

In [145]:
instruction_data=[]
for a in qa_data:
    for i, b in enumerate(a.split("\n\n")):
        i +=1
        q = b.split("Q"+str(i)+":")[1].split("\nA"+str(i)+":")[0].lstrip()
        ans = b.split("A"+str(i)+":")[1].split("\n\nQ"+str(i)+":")[0].lstrip()
        instruction_data.append({'instruction':q,'output':ans})

In [146]:
import random
for a in random.sample(instruction_data, 5):
    print(a)

{'instruction': '아이브 멤버들의 입사 순서는 어떻게 되나요?', 'output': '안유진(2016년) - 가을&장원영(2017년) - 레이(2018년) - 리즈&이서(2019년)'}
{'instruction': '이러한 콘셉트는 아이브의 이미지를 어떻게 변화시켰나요?', 'output': '이러한 콘셉트는 아이브의 이미지를 이전보다 더 강한 자신감과 매력을 가진 걸 크러시로 변화시켰습니다.'}
{'instruction': '아이브 공식 대형의 멤버 순서는 어떻게 바뀌었나요?', 'output': '아이브의 정규 1집 활동부터 멤버 순서가 레이 - 가을 - 장원영 - 안유진 - 이서 - 리즈 순서로 바뀌었습니다.'}
{'instruction': '아이브 멤버 중 가장 먼저 스타쉽엔터테인먼트에 입사한 멤버는 누구인가요?', 'output': '안유진(2016년)'}
{'instruction': '어떤 장소에서 발전하는 라이브 실력을 증명하고 있나요?', 'output': '앵 콜 무대와 기타 축제들에서의 라이브 무대를 통해 발전하는 라이브 실력을 증명하고 있습니다.'}


In [147]:
len(instruction_data)

248

### 덜 좋은 답변 만들기

참고자료를 주지 않고, 오로지 상상에만 맡겨서 데이터를 만들기!

그럼 자연스럽게 퀄러티가 떨어지는 답변을 얻을 수 있습니다!

ChatGPT는 아이브에 대해 알지 못하니까요^^

In [240]:
# 일단 위에서 만든 질문만 뽑아보자
questions = [ a['instruction'] for a in instruction_data]

In [241]:
questions[0]

'아이브는 어떤 소속사에 소속되어 있나요?'

In [242]:
len(questions)

248

In [253]:
# 질문을 10개 단위로 쪼개기
question_list = ["Q" + str((i%10)+1)+":" + q  for i, q in enumerate(questions)]

In [254]:
question_set =[]
for i in range(0, len(question_list), 10):
    question_set.append("\n".join(question_list[i:i+10]))

In [255]:
len(question_set)

25

In [256]:
print(question_set[0])

Q1:아이브는 어떤 소속사에 소속되어 있나요?
Q2:아이브는 몇 명으로 구성되어 있나요?
Q3:아이브의 데뷔일은 언제인가요?
Q4:아이브의 그룹명은 어디에서 유래되었나요?
Q5:아이브의 그룹명은 어떤 포부를 담고 있나요?
Q6:아이브는 어느 국적의 그룹인가요?
Q7:아이브의 멤버들은 누가 있나요?
Q8:아이브의 소속사에는 어떤 다른 그룹들이 있나요?
Q9:아이브는 데뷔 후 어떤 음악 활동을 펼칠 예정인가요?
Q10:아이브의 데뷔 앨범에는 어떤 곡들이 수록되어 있나요?


In [259]:
# 질문을 주고 지문 없이 바로 답변 물어보기!
PROMPT="### 주어진 질문들에 대해서 각각 답변해줘\n\
각 답변은 아래와 같은 형식으로 답변해줘.\n\
\n\
A1:답변1\n\
A2:답변2\n\
...\n\
A10:답변10\n\n\
### 질문들\n\
{questions}\n"

In [260]:
print(PROMPT.format_map({'questions':question_set[0]}))

### 주어진 질문들에 대해서 각각 답변해줘
각 답변은 아래와 같은 형식으로 답변해줘.

A1:답변1
A2:답변2
...
A10:답변10

### 질문들
Q1:아이브는 어떤 소속사에 소속되어 있나요?
Q2:아이브는 몇 명으로 구성되어 있나요?
Q3:아이브의 데뷔일은 언제인가요?
Q4:아이브의 그룹명은 어디에서 유래되었나요?
Q5:아이브의 그룹명은 어떤 포부를 담고 있나요?
Q6:아이브는 어느 국적의 그룹인가요?
Q7:아이브의 멤버들은 누가 있나요?
Q8:아이브의 소속사에는 어떤 다른 그룹들이 있나요?
Q9:아이브는 데뷔 후 어떤 음악 활동을 펼칠 예정인가요?
Q10:아이브의 데뷔 앨범에는 어떤 곡들이 수록되어 있나요?



In [261]:
bad_qa=[]
bad_qa_error=[]

In [262]:
for a in tqdm(question_set):
    try:
        response = openai.ChatCompletion.create(
            model='gpt-3.5-turbo',
            messages=[
                {'role':"system","content":"You are a helpful assistant."},
                {'role':"user","content":a},
                ],
            temperature=1,
        )
        bad_qa.append(response['choices'][0]['message']['content'])
        time.sleep(1)
    except Exception as e:
        bad_qa_error.append(a)
        print(title)
        print(e)

100%|██████████| 25/25 [07:31<00:00, 18.07s/it]


In [311]:
# 개행이 2개인 데이터도 있고, 1개인 데이터도 있다 -> 처리해줘야함
print(bad_qa[-7])

A1: 아이브의 멤버들이 2000년대 문화에 대해 알고 있는 정도는 제가 알 수 없습니다. 그러나 현재의 K-POP 그룹인 아이브 멤버들은 현재 세대인 Z세대에 속하므로 2000년대 문화에 대해 일부 알고 있을 것으로 예상됩니다.

A2: 데뷔 당시 최연소 아이돌 타이틀을 가진 아이브의 멤버는 리아입니다.

A3: 아이브의 멤버들의 연령은 1996년생인 어은지부터 2001년생인 리아까지로, 현재로서는 20대와 10대에 속하는 연령대입니다.

A4: 아이브의 멤버 원영과 이서가 슬라이드폰을 잘 사용하지 못하는 모습을 보인다고 하셨는데, 그들은 일반적으로 스마트폰 혹은 다른 유형의 터치스크린 휴대폰을 사용하는 것이 익숙할 것으로 예상됩니다.

A5: 원영과 이서를 제외한 다른 아이브 멤버들이 캔모아를 알고 있는지에 대해서는 알 수 없습니다. 그러나 그들은 동시대의 대중문화와 트렌드에 대해 알고 있을 가능성이 높습니다.

A6: 이서를 제외한 다른 아이브 멤버들은 마시마로에 대해 어떤 평가를 가지고 있는지에 대해서는 알 수 없습니다.

A7: 가을은 이서와 동료 멤버들과 비교해서 어떤 차이를 보이는지에 대해서는 알 수 없습니다. 추가적인 정보가 필요합니다.

A8: 가을이 드림 하이와 꽃보다 남자에 대해 알아볼 수 없다고 말한 것과 다른 멤버들이 이 작품들에 대해 잘 알고 있는지에 대해서는 알 수 없습니다.

A9: 2022년 4월 7일에 공개된 것은 제가 알아낼 수 없습니다. 구체적인 정보가 필요합니다.

A10: 에코세대는 1970년부터 1980년대 초에 태어난 세대를 말하며, Z세대는 1990년대 중반부터 2000년대 초에 태어난 세대를 말합니다.


In [320]:
bad_instruction_data=[]
for q, a in zip(question_set, bad_qa):
    q_s = q.replace("\n\n","\n").split("\n")
    a_s = a.replace("\n\n","\n").split("\n")

    # 한줄씩 자른 다음,  : 기준으로 잘라서 question과 answer만 뽑자!
    for qq, aa in zip(q_s, a_s):
        bad_instruction_data.append({'instruction':qq.split(":")[1], 'output': aa.split(":")[1]})

In [321]:
bad_instruction_data[0]

{'instruction': '아이브는 어떤 소속사에 소속되어 있나요?',
 'output': ' 아이브는 KONNECT 엔터테인먼트에 소속되어 있습니다.'}

In [324]:
bad_instruction_data[-1]

{'instruction': 'LOVE DIVE 활동 이전에는 모든 멤버가 어떤 종류의 활동명을 사용했나요?',
 'output': ' LOVE DIVE 활동 이전에는 멤버들은 다른 종류의 활동명을 사용하지 않았습니다.'}

In [323]:
len(bad_instruction_data)

248

### 좋은 질의응답과 나쁜 질의응답을 만들었으니, 이제 reward 모델의 학습을 위한 형태로 변환해 봅시다

In [328]:
reward_dataset=[]
# Prompt, 좋은 답변, 나쁜 답변 쌍으로 데이터 생성
for good, bad in zip(instruction_data, bad_instruction_data):
    reward_dataset.append({'prompt':good['instruction'],'chosen':good['output'], 'rejected':bad['output']})

In [329]:
len(reward_dataset)

248

In [330]:
reward_dataset[0]

{'prompt': '아이브는 어떤 소속사에 소속되어 있나요?',
 'chosen': '아이브는 스타쉽엔터테인먼트에 소속되어 있습니다.',
 'rejected': ' 아이브는 KONNECT 엔터테인먼트에 소속되어 있습니다.'}

In [331]:
reward_dataset[-1]

{'prompt': 'LOVE DIVE 활동 이전에는 모든 멤버가 어떤 종류의 활동명을 사용했나요?',
 'chosen': '성씨를 제외한 본명 및 예명',
 'rejected': ' LOVE DIVE 활동 이전에는 멤버들은 다른 종류의 활동명을 사용하지 않았습니다.'}

In [332]:
train_rm = []
test_rm = []
for a in reward_dataset:
    if random.random() < 0.1:
        test_rm.append(a)
    else:
        train_rm.append(a)

In [333]:
import json
with open("/content/drive/MyDrive/mygpt-lecture/data/stage2. RM/ive_train_rm.jsonl" , encoding= "utf-8",mode="w") as file:
	for i in train_rm: file.write(json.dumps(i, ensure_ascii=False) + "\n")

with open("/content/drive/MyDrive/mygpt-lecture/data/stage2. RM/ive_test_rm.jsonl" , encoding= "utf-8",mode="w") as file:
	for i in test_rm: file.write(json.dumps(i, ensure_ascii=False) + "\n")

# Stage 3 PPO 데이터 만들기, 질문만 있으면 됩니다!!

In [335]:
# 자신이 만든 사전학습 데이터 읽어오기!!

data=[]
for a in open("/content/drive/MyDrive/mygpt-lecture/data/stage1. domain_adaptive_pretraining/domain_adaptive_ive_pretrain.jsonl", 'r'):
    data.append(json.loads(a))

In [336]:
from tqdm import tqdm

In [337]:
question_only_data=[]
error_data=[]

In [338]:
# 최대한 다양한 질문을 만들어 낼 수 있는 프롬프트로 작성!

PROMPT="다음 주제와 내용을 보고 답변 할 수 있는 다양한 형태와 난이도의 창의적인 질문 20개를 만들어줘. n주제:{title}\n내용:{content}\n\
질문과 질문 사이는 반드시 ###을 붙여줘.\n\
답변의 형태는 다음과 같아.\n\
\n\
### Q1:질문1\n\
### Q2:질문2\n\
### Q3:질문3\n\
...\n\
### Q20:질문20\n\
"

for a in tqdm(data):

    title = a['instruction'].split("주제:")[-1]
    content = a['output']
    try:
        response = openai.ChatCompletion.create(
            model='gpt-3.5-turbo',
            messages=[
                {'role':"system","content":"You are a helpful assistant."},
                {'role':"user","content":PROMPT.format_map({'title':title, 'content':content})},
                ],
            temperature=1
        )
        question_only_data.append(response['choices'][0]['message']['content'])
    except Exception as e:
        error_data.append(a)
        print(title)
        print(e)

100%|██████████| 25/25 [09:16<00:00, 22.27s/it]


In [339]:
question_only_data[-1]

'Q1: IZ*ONE 멤버 중 LOVE DIVE 활동을 기점으로 활동명이 변경된 멤버는 누구인가요?\nA1: 안유진과 장원영\n\nQ2: 혈액형이 모두 다른 멤버는 누구인가요?\nA2: 가을, 안유진, 레이, 리즈, 장원영, 이서\n\nQ3: 멤버 중 몇 명이 서울공연예술고등학교를 중퇴 또는 졸업한 적이 있는가요?\nA3: 3명 (안유진, 레이, 장원영)\n\nQ4: IZ*ONE이 대중음악계 영향력 있는 전문가 투표에서 걸그룹 부문에서 몇 위를 차지했나요?\nA4: 3위\n\nQ5: 모든 멤버가 코로나19 확진 판정을 받은 적이 있는가요?\nA5: 예\n\nQ6: 2023년 잼버리 축제에서 멤버들은 어떤 헤어스타일을 할까요?\nA6: 데뷔 때와 비슷한 헤어스타일\n\nQ7: IZ*ONE에는 외동인 멤버가 있나요?\nA7: 아니요, 없습니다.\n\nQ8: 안유진, 레이, 원영은 각각 언니와 누나, 오빠와 형제 자매인가요?\nA8: 유진, 원영은 언니, 레이는 누나입니다.\n\nQ9: 리즈와 이서는 각각 형제와 자매가 있나요?\nA9: 리즈는 남동생이 있고, 이서는 여동생이 있습니다.\n\nQ10: LOVE DIVE 활동 이전에는 모든 멤버가 어떤 종류의 활동명을 사용했나요?\nA10: 성씨를 제외한 본명 및 예명'

In [349]:
prompt_dataset=[]
for q in question_only_data:
    q_s = q.replace("\n\n","\n").split("\n")

    # 한줄씩 자른 다음,  : 기준으로 잘라서 question과 answer만 뽑자!
    for qq in q_s:
        prompt_dataset.append({'prompt':qq.split(":")[1].lstrip()})

In [350]:
len(prompt_dataset)

500

In [352]:
for a in random.sample(prompt_dataset, 3):
    print(a)

{'prompt': '아이브의 멤버들이 가장 자주 하는 취미나 특기는 무엇일까요?'}
{'prompt': '이 그룹의 멤버들 중 가을과 다른 멤버들 사이에 어떤 차이점이 있나요?'}
{'prompt': '"After LIKE"가 Spotify 글로벌 차트에서 23위에 올랐다는 사실은 어떤 의미를 가지나요?'}


In [353]:
import random
train=[]
test=[]
for a in prompt_dataset:
    if random.random() < 0.1:
        test.append(a)
    else:
        train.append(a)

In [354]:
import json
with open("/content/drive/MyDrive/mygpt-lecture/data/stage3. PPO/ive_train_ppo.jsonl" , encoding= "utf-8",mode="w") as file:
	for i in train: file.write(json.dumps(i, ensure_ascii=False) + "\n")

with open("/content/drive/MyDrive/mygpt-lecture/data/stage3. PPO/ive_test_ppo.jsonl" , encoding= "utf-8",mode="w") as file:
	for i in test: file.write(json.dumps(i, ensure_ascii=False) + "\n")